In [ ]:
%pip install -r requirements.txt

### Configuring the Language Model (LLM)

Here, we define the configuration for the language model we want to use. This dictionary (`gemma`) specifies:
- **model**: The specific model to use (`gemma-2-2b-it`), along with its configuration identifier.
- **base_url**: The URL of the local server hosting the model API.
- **api_key**: The API key needed for authentication with the model server.
- **price**: An optional field to specify the cost per 1,000 tokens (for both input and output tokens).
- **cache_seed**: Set to `None` to disable caching, ensuring that each request is processed freshly without using previous results.

In [1]:
gemma = {
    "config_list": [
        {
            "model": "lmstudio-community/gemma-2-2b-it-GGUF/gemma-2-2b-it-Q4_K_M.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
            "price": [0.001, 0.001],  # Add custom pricing here (per 1k prompt/completion tokens)
        },
    ],
    "cache_seed": None,  # Disable caching.
}
verbose = False

In [3]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.reasoning_agent import ReasoningAgent, ThinkNode, visualize_tree

reason_agent = ReasoningAgent(
    name="reason_agent",
    system_message="answer math questions",
    llm_config=gemma,
    verbose=verbose,
    beam_size=1,  # using beam size 1 for special case of O1-style reasoning.
    max_depth=3,
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    max_consecutive_auto_reply=10,
)

In [5]:
question = "Alice has 3 brothers and she also has 2 sisters. How many sister does Alices brother has?"

def last_meaningful_msg(sender, recipient, summary_args):
    import warnings

    if sender == recipient:
        return "TERMINATE"

    summary = ""
    chat_messages = recipient.chat_messages[sender]

    for msg in reversed(chat_messages):
        try:
            content = msg["content"]
            if isinstance(content, str):
                summary = content.replace("TERMINATE", "")
            elif isinstance(content, list):
                # Remove the `TERMINATE` word in the content list.
                summary = "\n".join(
                    x["text"].replace("TERMINATE", "") for x in content if isinstance(x, dict) and "text" in x
                )
            if summary.strip().rstrip():
                return summary
        except (IndexError, AttributeError) as e:
            warnings.warn(f"Cannot extract summary using last_msg: {e}. Using an empty str as summary.", UserWarning)
    return summary

![](https://ag2ai.github.io/ag2/assets/images/reasoningagent_2-f550949b23619bca73bd10e0cc24e3dc.png)

In [6]:
ans = user_proxy.initiate_chat(reason_agent, message=question, summary_method=last_meaningful_msg)

user_proxy (to reason_agent):

Alice has 3 brothers and she also has 2 sisters. How many sister does Alices brother has?

--------------------------------------------------------------------------------
reason_agent (to tot_thinker):

# Question: Alice has 3 brothers and she also has 2 sisters. How many sister does Alices brother has?
---
What are the possible next steps?

--------------------------------------------------------------------------------
tot_thinker (to reason_agent):

**Reflection**
*The question is a bit of a trick, relying on wordplay to confuse the listener.  It seems like it's trying to mislead by focusing on siblings rather than defining if Alice and her brothers are inclusive of Alice herself. * 

**Possible Options:**
Option 1: Clarify whether the focus is on Alice and her siblings or all of their combined total, thereby providing context for the question.
Option 2:  Ask Alice a clarifying question to understand if she's referring to only her own set of siblings 

KeyboardInterrupt: 

In [ ]:
# After running a query, visualize the reasoning tree
print(ans.summary)
visualize_tree(reason_agent._root)

# Tree of Toughts

![](tree_of_thoughts.png)

In [ ]:
reason_agent = ReasoningAgent(
    name="reason_agent",
    llm_config=gemma,
    verbose=verbose,
    beam_size=3,
    max_depth=3,
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    max_consecutive_auto_reply=10,
)

In [ ]:
ans = user_proxy.initiate_chat(
    reason_agent,
    message="Alice has 3 brothers and she also has 2 sisters. How many sister does Alices brother has?",
    summary_method=last_meaningful_msg,
)